<div id="TOP" />
<a href="#END" > END of Notebook (filter) </a>

In [1]:
# -- CONFIG ------------------------------------
LAB_TAG=${PWD##*/} 
LAB_NAME="TFI_Azure_Lab8b. Importing Foreign Resources"
LAB_WEIGHT=82
LAB_MODE="Terraform"
# ----------------------------------------------

# NB=""; cd ~/nb/2021-Terraform/content/tf-labs/XXXX; set +x
DIE() { echo "[$PWD] Error: $*"; banner "ERROR"; exit 1; }
NBTOOL_RC=~/scripts/nbtool.rc; [ ! -f $NBTOOL_RC ] && DIE "No such script as $NBTOOL_RC"
echo "==== Reading nbtool functions from '$NBTOOL_RC' ===="; . $NBTOOL_RC
EXCL_FN_INIT_NOTEBOOK
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

==== Reading nbtool functions from '/home/student/scripts/nbtool.rc' ====
TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.

******** Tue 10 Jan 2023 03:55:53 PM UTC: Notebook initialized on tf TF-1.3.7

NB_DIR=/home/student/src/mjbright.tf-scenarios-private/ServeU

# --INCLUDE--SECTION--

In [2]:
EXCL_FN_LAB_ENV

2023-Jan-10:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

## Background:

In this lab, we will see how we can import AWS resources which were not created by Terraform itself so that Terraform can be subsequently used to manage these resources.

**Note**: Be careful not to manage resource both *outside* and *inside* Terraform

We will look at importing resources using
- the ```terraform import``` command

Tasks.
- As before, create a new directory lab 8b, and create three files, resources.tf, main.tf and vars.tf.

## Tasks

## 1. Importing a resource using ```terraform import```

### First Create a new lab directory and move to that directory


In [3]:
mkdir -p ~/labs/lab8b

cd       ~/labs/lab8b

#### First we will check if there are any instances running

Use the command ```/usr/local/bin/get_instances.sh -ir``` to determine if there are already running instances.

If there are any you may wish to return to previous lab directories and perform ```terraform destroy``` to delete these resources.

The important thing is to know what instances if any are currently running

You should have no Terraform managed resources in the current workspace, verify this

In [4]:
/usr/local/bin/get_instances.sh -ir

Instance data written to /home/student/tmp/get_instances/get_instances.1594593.json

--[count:0]-- All running instances -----------------


: 1

In [5]:
terraform show

No state.


#### Launch a new instance using the aws client

Launch a new aws_instance using the aws cli client using the following command:

In [6]:
aws ec2 run-instances --count 1  --tag-specifications \
    'ResourceType=instance,Tags=[{Key=aws-cli,Value=true}]' \
    --image-id ami-0e42deec9aa2c90ce --instance-type t2.micro

{
    "Groups": [],
    "Instances": [
        {
            "AmiLaunchIndex": 0,
            "ImageId": "ami-0e42deec9aa2c90ce",
            "InstanceId": "i-030a9593c957b7aee",
            "InstanceType": "t2.micro",
            "LaunchTime": "2023-01-10T15:58:08.000Z",
            "Monitoring": {
                "State": "disabled"
            },
            "Placement": {
                "AvailabilityZone": "us-west-1c",
                "GroupName": "",
                "Tenancy": "default"
            },
            "PrivateDnsName": "ip-172-31-28-165.us-west-1.compute.internal",
            "PrivateIpAddress": "172.31.28.165",
            "ProductCodes": [],
            "PublicDnsName": "",
            "State": {
                "Code": 0,
                "Name": "pending"
            },
            "StateTransitionReason": "",
            "SubnetId": "subnet-01f855549f3efdd85",
            "VpcId": "vpc-0c4ad4047839bc08f",
            "Architecture": "x86_64",
            "BlockD

**Note**: that we added a tag to identify this as a non-Terraform resource

**Note**: You may need to wait a minute for the VM to be visible ( in the running state )

then verify the running instances again using

In [7]:
/usr/local/bin/get_instances.sh -ir

Instance data written to /home/student/tmp/get_instances/get_instances.1594992.json

--[count:0]-- All running instances -----------------


: 1

In [8]:
/usr/local/bin/get_instances.sh -ir

Instance data written to /home/student/tmp/get_instances/get_instances.1595634.json

--[count:1]-- All running instances -----------------
{"id":"i-030a9593c957b7aee","image":"ami-0e42deec9aa2c90ce","state":"running","Tags":[{"Key":"aws-cli","Value":"true"}],"LaunchTime":"2023-01-10T15:58:08.000Z"}


: 1

Again, you should have no Terraform managed resources in the current workspace, verify this


#### Create a resource file for the running instance (not yet managed by Terraform)

We can create a simple terraform resource to be able to import this instance so it can be managed by terraform.

Create a new file import.tf with the **following form** with the **appropriate values** for the instance **you created** earlier:

```hcl
resource "aws_instance" "imported_item" {
    ami = "YOUR_IMAGE_ID"
    instance_type = "YOUR_TYPE"
}
```

In [9]:
#EXCLUDE

cat > import.tf <<EOF
resource "aws_instance" "imported_item" {
    ami           = "ami-0e42deec9aa2c90ce"
    instance_type = "t2.micro"
}
EOF

#. ~/scripts/nbtool.rc

SET_VAR AWS_ID $( /usr/local/bin/get_instances.sh -ir |
                grep -m 1 '"id":' |  sed -e 's/{"id":"//' -e 's/".*//' )
#echo $XX_AWS_ID
#terraform import aws_instance.imported_item $XX_AWS_ID

# TODO: FIX var setting ... do we need XX_VAR_AWS_ID in code cell ??

Instance data written to /home/student/tmp/get_instances/get_instances.1595848.json
Correcting var name 'AWS_ID' to '__AWS_ID'
VAR __AWS_ID="i-030a9593c957b7aee"


You can now import your resource - use your instance id from the above output, e.g. *"i-06817c3963d2716c9"* in the command below:

In [10]:
#EXCLUDE

# Let them think for themselves about the terraform init !!

terraform init


Initializing the backend...

Initializing provider plugins...
- Finding latest version of hashicorp/aws...
- Installing hashicorp/aws v4.49.0...
- Installed hashicorp/aws v4.49.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [11]:
terraform import aws_instance.imported_item $__AWS_ID

aws_instance.imported_item: Importing from ID "i-030a9593c957b7aee"...
aws_instance.imported_item: Import prepared!
  Prepared aws_instance for import
aws_instance.imported_item: Refreshing state... [id=i-030a9593c957b7aee]

Import successful!

The resources that were imported are shown above. These resources are now in
your Terraform state and will henceforth be managed by Terraform.



### Verify that terraform has imported the resource

Now verify that you can see the instance using 
- Looking at the newly created ```terraform.tfstate``` file
- Using the ```terraform show``` and ```terraform state list``` commands

In [12]:
#EXCLUDE

echo terraform show:
terraform show

echo
echo terraform state list:
terraform state list

terraform show:
# aws_instance.imported_item:
resource "aws_instance" "imported_item" {
    ami                                  = "ami-0e42deec9aa2c90ce"
    arn                                  = "arn:aws:ec2:us-west-1:816376574968:instance/i-030a9593c957b7aee"
    associate_public_ip_address          = true
    availability_zone                    = "us-west-1c"
    cpu_core_count                       = 1
    cpu_threads_per_core                 = 1
    disable_api_stop                     = false
    disable_api_termination              = false
    ebs_optimized                        = false
    get_password_data                    = false
    hibernation                          = false
    id                                   = "i-030a9593c957b7aee"
    instance_initiated_shutdown_behavior = "stop"
    instance_state                       = "running"
    instance_type                        = "t2.micro"
    ipv6_address_count                   = 0
    ipv6_addresses            

### Reflect on this

Think about what we just did, we

- created an *AWS EC2 instance* without using Terraform

- we then created a dummy *import.tf* partially describing the resource, containing just
  - the *ami_id*
  - the *instance_type*
  
- we then ran the ```terraform import``` command referencing the instance *id* of the EC2 instance

Once we performed these steps the resource has now been imported into the terraform state.

We can use this mechanism to start to manage infrastructure resources previously generated outside of *Terraform*.

**NOTE:** Note also that this is not very functional, there are a lot of manual steps to take to import a single resource - but 3rd-party tools exist ...

#### Now we can destroy the instance

You succeeded, you can now cleanup using ```terraform destroy```

Remove, or rename import.tf as import.tf.bak

In [13]:
#EXCLUDE
terraform destroy -auto-approve

aws_instance.imported_item: Refreshing state... [id=i-030a9593c957b7aee]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # aws_instance.imported_item will be destroyed
  - resource "aws_instance" "imported_item" {
      - ami                                  = "ami-0e42deec9aa2c90ce" -> null
      - arn                                  = "arn:aws:ec2:us-west-1:816376574968:instance/i-030a9593c957b7aee" -> null
      - associate_public_ip_address          = true -> null
      - availability_zone                    = "us-west-1c" -> null
      - cpu_core_count                       = 1 -> null
      - cpu_threads_per_core                 = 1 -> null
      - disable_api_stop                     = false -> null
      - disable_api_termination              = false -> null
      - ebs_optimized                        = false -> null
      -

### Verify that the terraform state is empty

Check the ```terraform.tfstate``` file and run ```terraform state list``` to verify that we have now destroyed the resource.

### Note that independently of Terraform the VM instance is nevertheless present in AWS but ```Terminating```

Note that the ```/usr/local/bin/get_instances.sh -ir``` command shows ```Running``` instances only ... the previously destroyed instance may still be present but in the ```Terminated``` or other state-*

You can verify none ```Running``` instances using either of the commands
- ```/usr/local/bin/get_instances.sh```
- or ```aws ec2 describe-instances```



# **Stretch Goal**: Import other resource types


- Try now to import other resource types, such as an S3 bucket, using *terraform import*' and/or '*terraforming*' tools

In [14]:
EXCL_FN_LAB_ENV

2023-Jan-10:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg) -->
<img src="../images/LOGO_v2_CROPPED.jpg" width="200" />

# --EXCLUDE--SECTION--

## **NOTE:** Save notebook first

<div id="END" /> <a href="#TOP" > TOP OF PAGE </a>  <!-- must appear after EXCLUDE pragma !! -->

In [ ]:
NB_DIR=~/nb/2022-Terraform/content/tf-labs/8b.ImportingResources/
cd $NB_DIR

. ~/scripts/nbtool.rc
pwd
EXCL_FN_FILTER_NOTEBOOK $NB

bash: cd: /home/student/nb/2022-Terraform/content/tf-labs/8b.ImportingResources/: No such file or directory
TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
/home/student/labs/lab8b

==== Filtering current notebook [/home/student/src/mjbright.tf-scenarios-private/